# What 

This is related to #269.

To do recommendation exrtaction I need to complete a few tasks:

- [ ] Web scraping of TSB for its recommendations
- [ ] Safety action section extraction for ATSB and TSB (depending on if TSB recommendation dataset is in fact complete)
- [ ] Safety action section parsing to gedt the recommendations

In [ ]:
import importlib

import engine.gather.WebsiteScraping as WebsiteScraping

import pandas as pd

importlib.reload(WebsiteScraping)

# Web scraping of TSB

Firstly I need to scrape the recommendations of the TSB website.

This will be done in the `WebsiteScraping` module.

I am not sure if this recommendations dataset is complete or not.

In [ ]:
importlib.reload(WebsiteScraping)

recommendation_scraper = WebsiteScraping.TSBRecommendationsScraper('tsb_recommendations', refresh=True)

recommendation_scraper.extract_recommendations_from_website()

In [ ]:
tsb_recommendations = pd.read_pickle('tsb_recommendations')
tsb_recommendations

How does the current recommendations looks like

In [ ]:
taic_recommendations =  pd.read_pickle('../../output/recommendations.pkl')

example_taic_recommendation = taic_recommendations.loc[213, 'recommendations']
example_taic_recommendation


After looking through the recommendations. I think that they are a compelte amount. Even if there are some larger gaps.

I have spot checked the rail and the gaps there seem to check out as the none of the reports in that few years had any recommendations.